In [1]:
#import tensorflow as tf

In [2]:
import h5py as h5

In [3]:
#tf.reset_default_graph()


# modelDir='knee_trained_MoDL'

# loadChkPoint=tf.train.latest_checkpoint(modelDir)

In [4]:
with h5.File('/home/hpc/iwbi/iwbi009h/CIP_-MoDL_vs_VarNet-/test/file_brain_AXT1_201_6002695.h5','r') as f:
    print(f.keys())
    print(f['reconstruction_rss'])
    print(f['kspace'])
    with h5.File('test.h5', 'w') as target:
        target.create_dataset('atb', data = f['reconstruction_rss'])
        target.create_dataset('org', data = f['kspace'])
        print(target.keys())

<KeysViewHDF5 ['ismrmrd_header', 'kspace', 'reconstruction_rss']>
<HDF5 dataset "reconstruction_rss": shape (12, 320, 320), type "<f4">
<HDF5 dataset "kspace": shape (12, 20, 640, 320), type "<c8">
<KeysViewHDF5 ['atb', 'org']>


In [5]:
with h5.File('knee_demo_data.h5','r') as f:
    print(f.keys())
    print(f['atb'])
    print(f['org'])
    print(f['mask'])

<KeysViewHDF5 ['atb', 'csm', 'mask', 'org']>
<HDF5 dataset "atb": shape (1, 640, 368), type "<c8">
<HDF5 dataset "org": shape (1, 640, 368), type "<c8">
<HDF5 dataset "mask": shape (1, 640, 368), type "<c8">


In [6]:
with h5.File('demoImage.hdf5','r') as f:
    print(f.keys())
    print(f['tstMask'])
    print(f['tstCsm'])
    print(f['tstOrg'])

<KeysViewHDF5 ['tstCsm', 'tstMask', 'tstOrg']>
<HDF5 dataset "tstMask": shape (1, 256, 232), type "|i1">
<HDF5 dataset "tstCsm": shape (1, 12, 256, 232), type "<c8">
<HDF5 dataset "tstOrg": shape (1, 256, 232), type "<c8">


In [7]:
# From the original  MODL's dataset: 
# <KeysViewHDF5 ['trnCsm', 'trnMask', 'trnOrg', 'tstCsm', 'tstMask', 'tstOrg']>

# ... 
# <HDF5 dataset "trnCsm": shape (360, 12, 256, 232), type "<c8">
# <HDF5 dataset "trnMask": shape (360, 256, 232), type "|i1">
# <HDF5 dataset "trnOrg": shape (360, 256, 232), type "<c8">
# <HDF5 dataset "tstOrg": shape (164, 256, 232), type "<c8">
# <HDF5 dataset "tstMask": shape (164, 256, 232), type "|i1">
# >>> 


In [8]:
# import some libraries
import os,time
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import numpy as np
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm
import supportingFunctions as sf
import modelV2 as mm

tf.compat.v1.disable_eager_execution()

tf.compat.v1.reset_default_graph()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True

#--------------------------------------------------------------
#% SET THESE PARAMETERS CAREFULLY
nLayers=5
epochs=50
batchSize=1
gradientMethod='AG'
K=1
sigma=0.01
restoreWeights=False
#%% to train the model with higher K values  (K>1) such as K=5 or 10,
# it is better to initialize with a pre-trained model with K=1.
if K>1:
    restoreWeights=True
    restoreFromModel='04Jun_0243pm_5L_1K_100E_AG'

if restoreWeights:
    wts=sf.getWeights('savedModels/'+restoreFromModel)
#--------------------------------------------------------------------------
#%%Generate a meaningful filename to save the trainined models for testing
print ('*************************************************')
start_time=time.time()
saveDir='savedModels/'
cwd=os.getcwd()
directory=saveDir+datetime.now().strftime("%d%b_%I%M%P_")+ \
 str(nLayers)+'L_'+str(K)+'K_'+str(epochs)+'E_'+gradientMethod

if not os.path.exists(directory):
    os.makedirs(directory)
sessFileName= directory+'/model'


#%% save test model
tf.compat.v1.reset_default_graph()

csmT = tf.compat.v1.placeholder(tf.complex64,shape=(None,12,256,232),name='csm')
maskT= tf.compat.v1.placeholder(tf.complex64,shape=(None,256,232),name='mask')
atbT = tf.compat.v1.placeholder(tf.float32,shape=(None,256,232,2),name='atb')

out=mm.makeModel(atbT,csmT,maskT,False,nLayers,K,gradientMethod)
predTst=out['dc'+str(K)]
predTst=tf.identity(predTst,name='predTst')
sessFileNameTst=directory+'/modelTst'


*************************************************
Instructions for updating:
Use fn_output_signature instead


/home/hpc/iwbi/iwbi009h/CIP_-MoDL_vs_VarNet-/modl/modelV2.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  xbn=tf.compat.v1.layers.batch_normalization(x,training=trainning,fused=True,name='BN')


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [12]:

saver=tf.compat.v1.train.Saver()
with tf.compat.v1.Session(config=config) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    savedFile=saver.save(sess, sessFileNameTst,latest_filename='checkpointTst')
print ('testing model saved:' +savedFile)
#%% read multi-channel dataset
trnOrg,trnAtb,trnCsm,trnMask=sf.getData('training')
trnOrg,trnAtb=sf.c2r(trnOrg),sf.c2r(trnAtb)

#%%
tf.compat.v1.reset_default_graph()
csmP = tf.compat.v1.placeholder(tf.complex64,shape=(None,None,None,None),name='csm')
maskP= tf.compat.v1.placeholder(tf.complex64,shape=(None,None,None),name='mask')
atbP = tf.compat.v1.placeholder(tf.float32,shape=(None,None,None,2),name='atb')
orgP = tf.compat.v1.placeholder(tf.float32,shape=(None,None,None,2),name='org')


#%% creating the dataset
nTrn=trnOrg.shape[0]
nBatch= int(np.floor(np.float32(nTrn)/batchSize))
nSteps= nBatch*epochs

trnData = tf.data.Dataset.from_tensor_slices((orgP,atbP,csmP,maskP))
trnData = trnData.cache()
trnData=trnData.repeat(count=epochs)
trnData = trnData.shuffle(buffer_size=trnOrg.shape[0])
trnData=trnData.batch(batchSize)
trnData=trnData.prefetch(5)
iterator=tf.compat.v1.data.make_initializable_iterator(trnData)
orgT,atbT,csmT,maskT = iterator.get_next('getNext')

#%% make training model

out=mm.makeModel(atbT,csmT,maskT,True,nLayers,K,gradientMethod)
predT=out['dc'+str(K)]
predT=tf.identity(predT,name='pred')
loss = tf.reduce_mean(input_tensor=tf.reduce_sum(input_tensor=tf.pow(predT-orgT, 2),axis=0))
tf.compat.v1.summary.scalar('loss', loss)
update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)

with tf.compat.v1.name_scope('optimizer'):
    optimizer = tf.compat.v1.train.AdamOptimizer()
    gvs = optimizer.compute_gradients(loss)
    capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
    opToRun=optimizer.apply_gradients(capped_gvs)


#%% training code


print ('training started at', datetime.now().strftime("%d-%b-%Y %I:%M %P"))
print ('parameters are: Epochs:',epochs,' BS:',batchSize,'nSteps:',nSteps,'nSamples:',nTrn)

saver = tf.compat.v1.train.Saver(max_to_keep=100)
totalLoss,ep=[],0
lossT = tf.compat.v1.placeholder(tf.float32)
lossSumT = tf.compat.v1.summary.scalar("TrnLoss", lossT)

with tf.compat.v1.Session(config=config) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    if restoreWeights:
        sess=sf.assignWts(sess,nLayers,wts)

    feedDict={orgP:trnOrg,atbP:trnAtb, maskP:trnMask,csmP:trnCsm}
    sess.run(iterator.initializer,feed_dict=feedDict)
    savedFile=saver.save(sess, sessFileName)
    print("Model meta graph saved in::%s" % savedFile)

    writer = tf.compat.v1.summary.FileWriter(directory, sess.graph)
    for step in tqdm(range(nSteps)):
        try:
            tmp,_,_=sess.run([loss,update_ops,opToRun])
            totalLoss.append(tmp)
            if np.remainder(step+1,nBatch)==0:
                ep=ep+1
                avgTrnLoss=np.mean(totalLoss)
                lossSum=sess.run(lossSumT,feed_dict={lossT:avgTrnLoss})
                writer.add_summary(lossSum,ep)
                totalLoss=[] #after each epoch empty the list of total loos
        except tf.errors.OutOfRangeError:
            break
    savedfile=saver.save(sess, sessFileName,global_step=ep,write_meta_graph=True)
    writer.close()

end_time = time.time()
print ('Trianing completed in minutes ', ((end_time - start_time) / 60))
print ('training completed at', datetime.now().strftime("%d-%b-%Y %I:%M %P"))
print ('*************************************************')


testing model saved:savedModels/10Feb_1223pm_5L_1K_50E_AG/modelTst
Reading the data. Please wait...
Elapsed time: 35.597615 seconds.

Successfully read the data from file!
Now doing undersampling....
Elapsed time: 111.887410 seconds.

Successfully undersampled data!
Instructions for updating:
Colocations handled automatically by placer.
training started at 10-Feb-2023 12:58 pm
parameters are: Epochs: 50  BS: 1 nSteps: 18000 nSamples: 360
Model meta graph saved in::savedModels/10Feb_1223pm_5L_1K_50E_AG/model


  0%|          | 56/18000 [05:38<30:09:10,  6.05s/it]


KeyboardInterrupt: 